# Lab 3: Query DSL

En esta práctica vamos a profundizar en los distintos operadores que podemos utilizar en con la DSL de consulta del Elasticserach:

* match
* term
* wildcard
* date
* geopoint
* bool
* range
* join

Las sentencias de este notebook las ejecutaremos sobre Kibana en la sección Dev Tools. Para acceder a Kibana entra en la siguiente URL: http://127.0.0.1:5601

Para acceder a la consola de Dev Tools entra en menú > Management > Dev Tools

Para ilustrar los diferentes tipos de consulta vamos a generar un índice que contenga información sobre libros. Los documentos que indexaremos en elasticsearch contienen los siguientes campos:

* title
* authors
* summary
* publish_date
* num_reviews
* publisher 


## 1. Creación de índice e insercción de datos

Antes de empezar a hacer las consultas vamos a crear el índcice book con su mapping type correspondiente.
Crearemos el índice con 3 particiones o shards y con una réplica ya que sólo tenemos un nodo de Elasticsearch.

* El campo title lo crearemos como un multi-field donde indexaremos el título como un campo text y un campo de tipo keyword, lo que nos permitirá ejecutar búsquedas full-text y filtrado de términos exactos sobre este campo.
* Los campos authors y publisher los vamos a crear como campos de tipo keyword para poder hacer búsquedas exactas sobre los autores y las editoriales. Estimamos que no es necesario hacer búsqueda full-text sobre estos campos.
* El campo summary lo crearemos como tipo text para poder hacer búsquedas de tipo full-text sobre el texto resumen de cada libro.
* num_revies lo crearemos de tipo long.
* El campo publish_date lo crearemos de tipo Date y le indicaremos el formato adecuado para el conjunto de datos que diponemos. 

`
PUT book
{
  "settings" : {
    "index" : {
      "number_of_shards" : 3,
      "number_of_replicas" : 1
    }
  }, 
  "mappings": {
    "properties": {
      "title": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "authors": {
        "type": "keyword"
      },
      "summary": {
        "type": "text"
      },
      "publish_date":{
        "type": "date",
        "format": "yyyy-MM-dd"
      },
      "num_reviews": {
        "type": "long"
      },
      "publisher" : {
        "type": "keyword"
      }
    }
  }
}
`

1. Comprueba en la sección de Index Management de Kibana que el índice se ha creado correctamente.



### 1.1 Ejercicio: Insertar los datos

Para realizar esta insercción vamos a utilizar la Bulk API, una api que permite realizar varias operaciones simultáneas sobre los índices. 

`
POST /book/_bulk
{ "index": { "_id": 1 }}
{ "title": "Elasticsearch: The Definitive Guide", "authors": ["clinton gormley", "zachary tong"], "summary" : "A distibuted real-time and scalable search and analytics engine", "publish_date" : "2015-02-07", "num_reviews": 20, "publisher": "oreilly" }
{ "index": { "_id": 2 }}
{ "title": "Taming Text: How to Find, Organize, and Manipulate It", "authors": ["grant ingersoll", "thomas morton", "drew farris"], "summary" : "organize text using approaches such as full-text search , proper name recognition, clustering, tagging, information extraction, summarization and analytics", "publish_date" : "2013-01-24", "num_reviews": 23, "publisher": "manning" }
{ "index": { "_id": 3 }}
{ "title": "Elasticsearch Guide in Action", "authors": ["radu gheorge", "matthew lee hinman", "timothy Gatsby"], "summary" : "build scalable search applications using Elasticsearch without having to do complex low-level programming or understand advanced data science algorithms", "publish_date" : "2015-12-03", "num_reviews": 18, "publisher": "manning" }
{ "index": { "_id": 4 }}
{ "title": "Solr in Action", "authors": ["trey grainger", "timothy Gatsby"], "summary" : "Comprehensive guide to implementing a scalable search engine using Apache Solr and Elasticsearch", "publish_date" : "2014-04-05", "num_reviews": 23, "publisher": "manning" }
{ "index": { "_id": 5 }}
{ "title": "Solr and Elasticsearch in Action", "authors": ["radu gheorge", "trey grainger", "timothy Gatsby"], "summary" : "Comprehensive guide to implementing a scalable search engine using Apache Solr and Elasticsearch", "publish_date" : "2011-03-10", "num_reviews": 53, "publisher": "manning" }
`

### 1.2. Ejercicio: Incluir los siguientes libros ejecutando la Api Bulk

**title**: The Great Gatsby, authors: F. Scott Fitzgerald
**summary**: In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since
**publish_date**: 2009-02-07
**num_reviews**: 5
**publisher**: anagrama 

**title**: The Grapes of Wrath and Gatsby
**authors**: [John Steinbeck, Lucas Stuart] 
**summary**: "To the red country and part of the gray country of Oklahoma, the last rains came gently, and they did not cut the scarred earth."
**publish_date**: 2009-02-17
**num_reviews**: 90 
**publisher**: salamandra

**title**: Nineteen Eighty-Four
**authors**: George Orwell
**summary**: "It was a bright cold day in April, and the clocks were striking thirteen"
**publish_date**: 1995-06-17
**num_reviews**: 23 
**publisher**: salamandra

## 2. Consultas Full-text o Match Queries

Se utilizan cuando queremos hacer consultas utilizando los campos analizados de tipo text. Se ejecutan dentro del query context, por lo que devolveran los resultados ordenados por la relevancia del término que se está buscando en el documento. 

Buscan el término en el contenido del campo o campos sobre el que se busca. 

En este ejemplo los campos se han analizado con el analizador por defecto de Elasticsearch, si se quiere sacar el máximo partido a este tipo de búsquedas es necesario especificar correctamente los analizadores para estos campos. Esto lo veremos en otros ejecicios más adelante.

También se suelen llamar MATCH Queries puesto que es la familia de operadores que vamos a utilizar para buscar sobre estos campos: "match", "match_phrase", "multi_match", etc:


Si queremos buscar el término en un campo específico lo realizaremos utilizando el operador match de la siguiente forma:

`
GET book/_search
{
    "query": {
        "match": { "summary" : "Elasticsearch"}
    }
}
`

Si queremos buscar más de un término en un campo específico realizaremos la consulta de  las siguiente manera:

`
GET book/_search?pretty
{
    "query": {
        "match" : {
            "summary" : {
                "query" : "applications using Elasticsearch"
            }
        }
    }
}
`

1. ¿Los documentos que nos ha devuelto contienen exactamente esa frase en el campo "summary"?
2. ¿El documento con mejor "_score" es el que contiene la frase literal en el campo summary? 

Por defecto Elasticsearch analiza la frase que le pasamos a la query con el analizador por defecto y extraerá los tokens de esa frase, en este caso [applications, using, elasticsearch], por lo que devolverá los documentos que contengan al menos uno de esos tokens. Aplicará la función de scoring a los resultado y dará más puntuación a los documentos que contengan los tres tokens.

Si lo que queremos es buscar los tres tokens simultáneamente dentro del campo, entonces tendremos que indicar que concatene los tokens extraídos con el operador and (por defecto usa el operador or): 

`
GET book/_search?pretty
{
    "query": {
        "match" : {
            "summary" : {
                "query" : "applications using Elasticsearch",
                "operator" : "and"
            }
        }
    }
}
`

1. ¿Qué pasa si en el campo query de la consuta especifico la siguiente frase: "Elasticsearch using applications"? Crea la sentencia y ejecútala para ver el resultado.
2. ¿Importa el orden de los tokens en la frase de la consulta y en el texto del campo del documento analizado?

Si lo que queremos es buscar una frase en concreto entonces tendremos que utilizar el operador mach_phrase:

`
GET book/_search
{
    "query": {
        "match_phrase": {
            "summary": "applications using Elasticsearch"
        }
    }
}
`

1. ¿Qué pasa ahora si ejecutamos la sentencia anteriror con el valor "Elasticsearch using applications"? Crea la sentencia y ejecutala para ver el resultado.

Para hacer estas búsquedas tambien podemos utilizar la sintaxsis de búsqueda nativa de Lucene:
* https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html#query-string-syntax

Por ejemplo si queremos buscar el término Elasticsearch en todos los campos de tipo text del documento:

`
GET /book/_search?q="applications using Elasticsearch"
`

Si queremos especificar un campo concreto:

`
GET /book/_search?q=summary:Elasticsearch
`

También podemos usar operadores AND y OR:

`
GET /book/_search?q=(applications OR using OR Elasticsearch)
`

`
GET /book/_search?q=(applications AND using AND Elasticsearch)
`

Otra forma de ejecutar estas sentencias es utilizando el operador query_string:

`
GET /book/_search
{
    "query": {
      "query_string" : {
        "query": "(applications AND using AND Elasticsearch)"
      }
    }
}
`

### 2.1 Ejercicio: Realiza una búsqueda sobre todos los campos de tipo text que devuelva los documentos que tengan la palabra salamandra.


Si lo que quermos es buscar un texto en un conjunto de campos, en tonces utilizaremos el operador "multi_match":

`
GET /book/_search
{
  "query": {
    "multi_match" : {
      "query":    "applications using Elasticsearch", 
      "fields": [ "summary", "title" ] 
    }
  }
}
`
### 2.2 Ejercicio: Busca el texto "Gatsby" en los campos title y author

Por último si queremos que la búsqueda se comporte como una match_phrase entonces podemos especificarlo en el parámetro "type" de la query:

`
POST /book/_search
{
    "query": {
        "multi_match" : {
            "query": "applications using Elasticsearch",
            "fields": ["title", "summary"],
            "type": "phrase"
        }
    }
}
`

###  2.3 Ejercicio: Buscar los libros que tengan en el título o sumario la frase "Comprehensive scalable". Incluir el slop máximo para que aparezca un documento

Dado que estamos buscando en varios campos, es posible que desee aumentar las relevancia de un determinado campo. Por ejemplo aumentar las relevancia del campo de resumen por un factor de 3 para aumentar la importancia de encontrar el texto en dicho campo.

`
POST /book/_search
{
    "query": {
        "multi_match" : {
            "query" : "elasticsearch guide",
            "fields": ["title", "summary^3"]
        }
    },
    "_source": ["title", "summary", "publish_date"]
}
`

### 2.4. Ejercicio: Modifica la consulta del ejercicio 3.3 para dar más peso a la aparición en el campo título y que sólo aparezcan como resultado de la consulta los campos, title, summary y publisher

## 3. Term Queries

Las term queries se utilizan cuando queremos comprobar coincidencias exactas de términos y se utilizan para consultar campos de tipo keyword.

Ejemplos de uso son la búsqueda documentos con un email en concreto o un hostname específico, etc.

Por ejemplo si queremos buscar los documentos de una editorial en concreto, por ejemplo Manning, primero comprobamos que el campo "publisher" sea de tipo keyword y a continuación utilizamos el operador term para realizar la búsqueda, como al insertar los documentos indicamos el nombre de las editoriales en minúscula, utilizaremos el parámetro "case_insensitive" a true para que no tenga en cuenta las mayúsculas y minúsculas:

`
POST /book/_search
{
  "query": {
    "term": {
      "publisher": {
        "value": "Manning",
        "case_insensitive": true
      }
    }
  }
}
`

Si quisieramos buscar más de un valor en el campo utilizaríamos el operador "terms". Este operador tiene más limitaciones y no permite la búsqueda case insensitive:

`
POST /book/_search
{
  "query": {
    "terms": {
      "publisher": ["manning", "oreilly"]
    }
  }
}
`

Otra forma de realizar esta búsqueda es utilizar las Boolean Queries (que veremos en detalle más adelante en el notebook) que nos permiten combinar varias condiciones de búsqueda. En este caso vamos a utilizar el operador "should" que por ahora vamos a pensar que equivaldría a unir las dos condiciones con un or:

`
POST /book/_search
{
    "query": {
        "bool": {
            "should": [
              { "term": {
                  "publisher": {
                    "value": "oreilly",
                    "case_insensitive": true
                  }
                }
              },
              { "term": {
                  "publisher": {
                    "value": "Manning",
                    "case_insensitive": true
                  }
                }
              }
            ]
        }  
    }
}
`

1. ¿Qué escore tienen los resultados?

Como vemos nos ha permitido solucionar el problema del case sensitive, pero además nos permite por ejemplo darle más puntuación en el scooring a los documentos que tengan uno de los dos valores usando el parámetro "boost":

`
POST /book/_search
{
    "query": {
        "bool": {
            "should": [
              { "term": {
                  "publisher": {
                    "value": "oreilly",
                    "case_insensitive": true
                  }
                }
              },
              { "term": {
                  "publisher": {
                    "value": "Manning",
                    "case_insensitive": true,
                    "boost": 2.0
                  }
                }
              }
            ]
        }  
    }
}
`

## 4. Wildcards Queries y expresiones regulares

Las consultas de comodines nos permiten usar expresiones regulares para hacer las búsquedas sobre un campo.
* Si ejecutamos la consulta sobre un campo de tipo keyworkd macheará contra la cadena de texto entera. 
* Si ejecutamos la consulta sobre un campo de tipo text macheará contra cada uno de los tokens extraidos por el analizador del campo.

`
POST /book/_search
{
    "query": {
        "wildcard" : {
            "authors" : "t*"
        }
    }
}
`

Si queremos ver exactamente que porción del texto es la que ha macheado, por ejemplo tenemos loibros con varios autores y queremos saber exactamente que autor es el que ha macheado con la expresión regular, podemos usar la condición "highlight" que resalta el match de la query: 

`
POST /book/_search
{
    "query": {
        "wildcard" : {
            "authors" : "t*"
        }
    },
    "highlight": {
        "fields" : {
            "authors" : {}
        }
    }
}
`

### 4.1 Ejercicio: Busca los libros cuyo autor empiece por g y termine en 'orge'

## 5. Boolean Query

Las sentencias de tipo booleanas permiten anidar varias condciones de búsqueda sobre los documentos y con el nivel de profundidad que necesitemos.
Dependiendo del operador que utilicemos, la condición de búsqueda se tendrá en cuenta o no para la puntuación de los resultados de la sentencia.
Los operadores que disponemos son los siguientes:
* **"must"**: la condición debe(must) aparecer en los documentos y contribuye a la putuación.
* **"must_not"**: la condición no debe aparecer en los documento y NO contribuye a la puntuación, se ejecuta en el filter context.
* **"should"**: la condición debería(shoud) aparecer en los documentos y contribuye a la puntuación.
* **"filter"**: se compora igual que el must pero NO contribuye a la puntuación y se ejecuta en el filter context.

Vamos a ver algunos ejemplos e intentar entender como funcionan los operadores y como participan en la puntuación.


Primero vamos a buscar aquellos documentos cuyo título contenga el término Elasticsearch o Solr. Puesto que es cualquiera de los dos términos utilizaremos el operador "should":

`
POST /book/_search
{
    "query": {
        "bool": {
            "should": [
              { "match": { "title": "Elasticsearch" }},
              { "match": { "title": "Solr" }} 
            ]
        }
    }
}
`

1. ¿Cuál es el que tiene más puntuación?

Como vemos en el resultado el comando "should" interviene en la puntuación, dando más puntuación al documento que contiene ambos términos en el campo título.

Podemos ejecutar esta misma consulta utilizando el contexto de filtrado, es decir utilizando el operador "filter":

`
POST /book/_search
{
    "query": {
        "bool": {
            "filter": {
              "bool": { 
                "should": [
                  { "match": { "title": "Elasticsearch" }},
                  { "match": { "title": "Solr" }} 
                ]
              }
            }
        }
    }
}
`

1. ¿Cuál es el que tiene más puntuación?

En este caso todos tienen la misma puntuación, puesto que las condiciones expresadas en el contexto de filtrado no puntúan.


Volvemos a la sentencia inical de la sección, si en vez del operador "should" utilizamos el operador "must"

`
POST /book/_search
{
    "query": {
        "bool": {
            "must": [
              { "match": { "title": "Elasticsearch" }},
              { "match": { "title": "Solr" }} 
            ]
        }
    }
}
`

Sólo nos devuleve el documento que contiene en su título ambos términos.

Vamos a indicar ahora que excluya todos los documentos cuyo autor sea "Radu Gheorge" para ello utilizamos el operador "must_not".

`
POST /book/_search
{
    "query": {
        "bool": {
            "should": [
              { "match": { "title": "Elasticsearch" }},
              { "match": { "title": "Solr" }} 
            ],
            "must_not": { "match": {"authors": "radu gheorge" }}
        }
    }
}
`

### 5.1  Ejercicio: Buscar los libros de timothy Gatsby de marzo de 2014

## 6. Range query

El operador range permite buscar documentos entre rangos de valores. 
Por ejemplo si queremos buscar los libros que tienen reviews comprendidas entre 20 y 50 realizaríamos la siguiente operación:

`
POST /book/_search
{
    "query": {
        "range" : {
            "num_reviews": {
                "gte": "20",
                "lte": 50
            }
        }
    }
}
`

También podemos aplicar rangos a otros tipos de campos, como por ejemplo a los campos de tipo Date:

`
POST /book/_search
{
    "query": {
        "range" : {
            "publish_date": {
                "gte": "2010-01-01",
                "lte": "2020-01-01"
            }
        }
    }
}
`

### 6.1 Ejercicio: Realiza una consulta que encuentre los libros en los que aparezca en el título: "Elasticsearch Guide", que no sean del autor Mathew, junto con los que tengan 23 reviews

## 7. Odenación, proyección y paginación

La DSL de consulta nos permite también operar con los resultado devueltos por la consulta, como ordenar el resultado por uno o vrarios campos, sleccionar los campos del documento a devolver y/o paginar los resultados.

Primero vamos a ver como ordenar los resultados. Si queremos ordenar el resultado de buscar los libros que contengan en el título el término "Elasticsearch" por el número de reviews utilizamos el operador "sort" para hacerlo:

`
POST /book/_search
{
    "query": {
        "match" : {
            "title" : "Elasticsearch"
        }
    },
    "sort": { "num_reviews": {"order": "desc" } } 
}
`

1. ¿Qué pasa con la puntuación de los documentos?

Podemos ordernar por más de un campo:

`
POST /book/_search
{
    "query": {
        "match" : {
            "title" : "Elasticsearch"
        }
    },
    "sort": [
      { "publish_date": {"order": "asc" } },
      { "num_reviews": {"order": "desc" } }
    ]
}
`

Si sólo queremos que nos devuelva ciertos campos de documentos utilizaremos el operador "_source" para indicarle a Elsticsearch que campos queremos que nos devuelva:

`
POST /book/_search
{
    "query": {
        "match" : {
            "title" : "Elasticsearch"
        }
    },
    "sort": { "num_reviews": {"order": "desc" } },
    "_source": [ "title", "summary", "publish_date" ]
}
`

Elastic search siempre nos devuelve los datos paginados, es decir sólo nos devuelve por defecto los primeros 10 documentos que cumplen la condición de la búsqueda ordenado por defecto por la relevancia o puntuación de la consulta. Ya hemos visto como cambiar el orden de los documentos, para indicar el número de documentos que queremos que nos devuelva utilizamos el parámetro "size":


`
POST /book/_search
{
    "query": {
        "match" : {
            "title" : "Elasticsearch"
        }
    },
    "sort": { "num_reviews": {"order": "desc" } },
    "_source": [ "title", "summary", "publish_date" ],
    "size": 2
}
`

y si queremos indicar la página que queremos que nos devuelva tendremos que utilizar el parámetro "from" que indica el offset al partir del que buscar:

`
POST /book/_search
{
    "query": {
        "match" : {
            "title" : "Elasticsearch"
        }
    },
    "sort": { "num_reviews": {"order": "desc" } },
    "_source": [ "title", "summary", "publish_date" ],
    "from": 0,
    "size": 2
}
`

Si queremos obtener la siguiente página tendremos que cambiar el valor de from. El valor de size deveríamos mantenerlo para tener páginas siempre del mismo tamaño:

`
POST /book/_search
{
    "query": {
        "match" : {
            "title" : "Elasticsearch"
        }
    },
    "sort": { "num_reviews": {"order": "desc" } },
    "_source": [ "title", "summary", "publish_date" ],
    "from": 2,
    "size": 2
}
`

### 7.1 Ejercicio: Realiza una búsqueda sobre el campo summary del texto que devuelva como máximo 3 resultados y sólo devuelva los campos title, summary y num_reviews. Queremos marcar los aciertos (highligts) en el campo summary.

## 8. Join Queries

Los campos de tipo join nos permiten crear relaciones de tipo padre/hijo entre documentos del mismo índice. Utilizaremos las sentencias de tipo Join para recuperar los documentos relacionados, por ejemplo, los documentos que tengan un padre en concreto o que cumplan una condición o los documentos que tengan unos hijos que cumplan otra determinada condición.

Para ver este ejemplo creamos un nuevo índice que almacene información de los departamentos de una empresa y los empleados de dicha empresa. Crearemos un campo de tipo join para relacionar los documentos del índice estableciendo la relación de parentesco departamento/empleado, donde el departamento es el padre y empleado es el hijo. De esta forma podemos relacionar que empleados pertenencen a un departamento.

Tendremos por tanto dos tipos de documentos, aquellos que hacen referencia a los departamentos que sólo contendrán dos campos, dept_id y dept_name y un segundo tipo de documentos que hacen referencia a los empleados que sólo tendrán dos campos, employee_id y employee_name. 

Definimos el mapping que pueda almacenar los dós tipos de documentos que vamos a insertar junto con el campo doc_type de tipo join que nos permitirá expresar la relacción de parentesco entra ambos tipos de docimentos (department padre de employee: "department": "employee") y creamos el índice:

`
PUT department-employees
{
  "mappings": {
    "properties": {
      "dept_id": { "type": "keyword" },
      "dept_name": { "type": "keyword" },
      "employee_id": { "type": "keyword" },
      "employee_name": { "type": "keyword" },
      "doc_type": {
        "type": "join",
          "relations": {
            "department": "employee"
          }
        }
      }
    }
  }
}
`

Vamos a insertar unos pocos documentos con ifnormación de depatamentos, que serán documentos padre por tanto en el campo doc_type indicamos que son del tipo padre: "doc_type": "department".

`
PUT department-employees/_doc/d1
{
  "dept_id": "D001",
  "dept_name": "Finance",
  "doc_type": "department"
}
`

`
PUT department-employees/_doc/d2
{
  "dept_id": "D002",
  "dept_name": "HR",
  "doc_type": "department"
}
`

`
PUT department-employees/_doc/d3
{
  "dept_id": "D003",
  "dept_name": "IT",
  "doc_type": "department"
}
`

Ahora pasamos a insertar los documentos hijo. Para ello tenemos que indicar en el campo "doc_type" que son del tipo hijo y cuál es su documento padre, "doc_type": { "name": "employee", "parent": "d3" }. Puesto que los documentos hijo tienen que estar en el mismo shard que el padre, es necesario enrutar la operación al nodo donde se encuentre el documento padre, para ello usamos el parámetro routing:

`
PUT department-employees/_doc/e1?routing=d3
{
  "employee_id": "E001",
  "employee_name": "Sarah",
  "doc_type": {
    "name": "employee",
    "parent": "d3"
  }
}
`

`
PUT department-employees/_doc/e2?routing=d3
{
  "employee_id": "E002",
  "employee_name": "James",
  "doc_type": {
    "name": "employee",
    "parent": "d3"
  }
}
`

`
PUT department-employees/_doc/e3?routing=d2
{
  "employee_id": "E003",
  "employee_name": "Ben",
  "doc_type": {
    "name": "employee",
    "parent": "d2"
  }
}
`


Si quisieramos consultar cuales son todos los empleados del departamento de "IT" haríamos la siguiente join query indicando que queremos los documentos que tienen un padre ("has_parent") de tipo "department" ("parent_type": "department") cuyo "dept_name" es "IT" :

`
GET department-employees/_search
{
  "query": {
    "has_parent": {
      "parent_type": "department",
      "query": {
        "term": {
          "dept_name": { "value": "IT" }
        }
      }
    }
  }
}
`

Si quisiéramos saber en que departamentos trabaja Ben ejecutaríamos la siguiente join query indicando que queremos consultar los documentos que tienen como documentos hijo ("has_child") documentos de tipo "employee" cuyo "employee_name" es "Ben":

`
GET department-employees/_search
{
  "query": {
    "has_child": {
      "type": "employee",
      "query": {
        "term": { "employee_name": "Ben" }
      }
    }
  }
}
`

### 8.1 Ejercicio: Obten todos los empleado de los departamentos de Finanzas y Recursos Humanos